#### Вариант 3:
**"Анализ клиентов"**

- Создать DataFrame с информацией о клиентах
- Найти клиентов с наибольшим количеством заказов
- Рассчитать средний возраст по департаменту
- Построить отчет по активности

In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, LongType, StringType, IntegerType, DecimalType, TimestampType, BooleanType
from pyspark.sql.functions import col, sum, round, avg, max, count, count

spark = SparkSession.builder \
        .appName("Lab1_v3") \
        .master("local[*]") \
        .config("spark.sql.shuffle.partitions", 4) \
        .getOrCreate()

file_path = "../../data/simple_customers_10000.csv"

data_schema = StructType([
    StructField("id", LongType(), False),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("email", StringType(), True),
    StructField("phone", StringType(), True),
    StructField("city", StringType(), True),
    StructField("total_spent", DecimalType(10, 2), True),
    StructField("orders_count", IntegerType(), True),
    StructField("is_active", BooleanType(), True)
])

df_sales = spark.read.csv(
    file_path,
    header=True,
    schema=data_schema
)

df_sales.show(10)

+---+----------+---------+------+---+--------------------+------------+---------------+-----------+------------+---------+
| id|first_name|last_name|gender|age|               email|       phone|           city|total_spent|orders_count|is_active|
+---+----------+---------+------+---+--------------------+------------+---------------+-----------+------------+---------+
|  1| Александр|  Смирнов|     М| 26|александр.смирнов...|+79207072090|         Москва|     744.79|           7|    false|
|  2|   Алексей|    Попов|     М| 72|алексей.попов585@...|+79024670094|         Москва|   44613.65|          85|    false|
|  3|     Ольга| Сидорова|     Ж| 36|ольга.сидорова869...|+79977328028|Санкт-Петербург|   36088.95|          73|     true|
|  4|    Андрей|    Попов|     М| 71|андрей.попов762@m...|+79338904486|    Новосибирск|   42221.04|           6|    false|
|  5|     Ольга|  Иванова|     Ж| 68|ольга.иванова483@...|+79161652939|         Москва|   44796.65|          72|    false|
|  6|    Сергей|

In [11]:
# Найти клиентов с наибольшим количеством заказов
df_top_clients = df_sales.orderBy(col("orders_count"), col("total_spent"), ascending=False).limit(10)
df_top_clients.show()

+----+----------+---------+------+---+--------------------+------------+---------------+-----------+------------+---------+
|  id|first_name|last_name|gender|age|               email|       phone|           city|total_spent|orders_count|is_active|
+----+----------+---------+------+---+--------------------+------------+---------------+-----------+------------+---------+
|6190|    Андрей|    Попов|     М| 47|андрей.попов203@m...|+79397885226|Санкт-Петербург|   49014.98|         100|    false|
|4453|    Сергей| Кузнецов|     М| 34|сергей.кузнецов43...|+79251708283|         Москва|   48966.72|         100|    false|
|8982|    Сергей|   Петров|     М| 30|сергей.петров616@...|+79294576024|Санкт-Петербург|   48934.00|         100|     true|
|6995|  Светлана| Новикова|     Ж| 47|светлана.новикова...|+79726941706|Санкт-Петербург|   47273.78|         100|    false|
|8835|      Анна|  Иванова|     Ж| 19|анна.иванова741@m...|+79371431156|Санкт-Петербург|   47089.46|         100|     true|
|5749| А

In [12]:
# Рассчитать средний возраст по департаменту (просто рассчитаем средний возраст)
df_avg_age = df_sales.groupBy(col("city")).agg(
    round(avg(col("age")), 2).alias("AVG_AGE")
).orderBy(col("AVG_AGE").desc())
df_avg_age.show()

+---------------+-------+
|           city|AVG_AGE|
+---------------+-------+
|   Екатеринбург|  49.39|
|    Новосибирск|  49.24|
|         Москва|   49.0|
|         Казань|  48.99|
|Санкт-Петербург|  48.47|
+---------------+-------+



In [15]:
# Построить отчет по активности
df_report = df_sales.groupBy(col("city"), col("is_active")).agg(
    count("*").alias("cnt")
).orderBy(col("city"))

df_report.show()

+---------------+---------+----+
|           city|is_active| cnt|
+---------------+---------+----+
|   Екатеринбург|     true|1004|
|   Екатеринбург|    false| 994|
|         Казань|    false| 996|
|         Казань|     true|1026|
|         Москва|    false|1026|
|         Москва|     true| 954|
|    Новосибирск|    false|1001|
|    Новосибирск|     true| 962|
|Санкт-Петербург|     true|1021|
|Санкт-Петербург|    false|1016|
+---------------+---------+----+

